In [1]:
import pandas as pd
from sklearn.metrics import log_loss, brier_score_loss

In [30]:
odds_df = pd.read_csv("../../data/backtesting/backtest_odds.csv", parse_dates=["date"]).drop(columns=["sportsbook"])
odds_df["red_odds"] = odds_df["red_odds"].apply(lambda x: 100 / (x + 100) if x > 0 else -x / (-x + 100))
odds_df["blue_odds"] = odds_df["blue_odds"].apply(lambda x: 100 / (x + 100) if x > 0 else -x / (-x + 100))
odds_df["Bovada"] = odds_df["red_odds"] / (odds_df["red_odds"] + odds_df["blue_odds"])
odds_df = odds_df.drop(columns=["red_odds", "blue_odds"])
preds_df = pd.read_csv("../../model_files/lr/predictions.csv")
df = odds_df.merge(preds_df, on=["bout_id"]).rename(columns={"y_pred": "Model"})
df

,bout_id,event_id,date,red_win,Bovada,Model
0,c1356395d6b055d7,46effbd1135423c5,2017-01-15,0.0,0.489879,0.544421
1,ae803440d778a12b,46effbd1135423c5,2017-01-15,0.0,0.328185,0.227241
2,3f7684492c9df05e,46effbd1135423c5,2017-01-15,1.0,0.585987,0.678656
3,53c3565ee1d3411a,46effbd1135423c5,2017-01-15,0.0,0.390764,0.418130
4,8156479490877d08,46effbd1135423c5,2017-01-15,1.0,0.601770,0.727730
...,...,...,...,...,...,...
3955,5238f6470d0557fb,72c9c2eadfc3277e,2024-12-14,0.0,0.305344,0.379029
3956,7b1bc4ff776f12c1,72c9c2eadfc3277e,2024-12-14,0.0,0.735516,0.893173
3957,1a635a5e4551e7d5,72c9c2eadfc3277e,2024-12-14,1.0,0.772329,0.759408
3958,7521015554088962,72c9c2eadfc3277e,2024-12-14,1.0,0.351079,0.456889


In [16]:
temp = df.loc[(df["date"].dt.year == 2024) & (df["red_win"].notnull())].copy()
bovada_log_loss = log_loss(temp["red_win"], temp["Bovada"])
model_log_loss = log_loss(temp["red_win"], temp["Model"])
bovada_brier_score = brier_score_loss(temp["red_win"], temp["Bovada"])
model_brier_score = brier_score_loss(temp["red_win"], temp["Model"])

print(f"Model Log Loss: {model_log_loss:.6f}")
print(f"Bovada Log Loss: {bovada_log_loss:.6f}")
print(f"Delta: {model_log_loss - bovada_log_loss:.6f}")

print(f"Model Brier Score: {model_brier_score:.6f}")
print(f"Bovada Brier Score: {bovada_brier_score:.6f}")
print(f"Delta: {model_brier_score - bovada_brier_score:.6f}")

Model Log Loss: 0.586208
Bovada Log Loss: 0.583386
Delta: 0.002822
Model Brier Score: 0.199778
Bovada Brier Score: 0.198925
Delta: 0.000853


In [17]:
features = pd.read_pickle("../../data/features.pkl.xz")
features

,id,avg_knockdowns_scored_diff,cumulative_knockdowns_scored_diff,avg_knockdowns_scored_per_second_diff,cumulative_knockdowns_scored_per_second_diff,avg_knockdowns_scored_per_strike_landed_diff,cumulative_knockdowns_scored_per_strike_landed_diff,avg_knockdowns_scored_per_strike_attempted_diff,cumulative_knockdowns_scored_per_strike_attempted_diff,avg_knockdowns_scored_per_significant_strike_landed_diff,cumulative_knockdowns_scored_per_significant_strike_landed_diff,avg_knockdowns_scored_per_significant_strike_attempted_diff,cumulative_knockdowns_scored_per_significant_strike_attempted_diff,avg_knockdowns_scored_per_significant_strike_head_landed_diff,cumulative_knockdowns_scored_per_significant_strike_head_landed_diff,avg_knockdowns_scored_per_significant_strike_head_attempted_diff,cumulative_knockdowns_scored_per_significant_strike_head_attempted_diff,avg_ko_tko_landed_diff,cumulative_ko_tko_landed_diff,avg_ko_tko_landed_per_second_diff,cumulative_ko_tko_landed_per_second_diff,avg_ko_tko_landed_per_strike_landed_diff,cumulative_ko_tko_landed_per_strike_landed_diff,avg_ko_tko_landed_per_strike_attempted_diff,cumulative_ko_tko_landed_per_strike_attempted_diff,avg_ko_tko_landed_per_significant_strike_landed_diff,cumulative_ko_tko_landed_per_significant_strike_landed_diff,avg_ko_tko_landed_per_significant_strike_attempted_diff,cumulative_ko_tko_landed_per_significant_strike_attempted_diff,avg_ko_tko_landed_per_significant_strike_head_landed_diff,cumulative_ko_tko_landed_per_significant_strike_head_landed_diff,avg_ko_tko_landed_per_significant_strike_head_attempted_diff,cumulative_ko_tko_landed_per_significant_strike_head_attempted_diff,avg_total_strikes_landed_diff,cumulative_total_strikes_landed_diff,avg_total_strikes_landed_per_second_diff,cumulative_total_strikes_landed_per_second_diff,avg_total_strikes_accuracy_diff,expected_avg_total_strikes_accuracy_diff,cumulative_total_strikes_accuracy_diff,...,avg_event_attendance_change_diff,avg_event_occupancy_pct_diff,event_occupancy_pct_change_diff,avg_event_occupancy_pct_change_diff,avg_opp_wins_at_venue_diff,avg_opp_wins_at_venue_diff_diff,avg_opp_losses_at_venue_diff,avg_opp_losses_at_venue_diff_diff,avg_opp_win_rate_at_venue_diff,avg_opp_win_rate_at_venue_diff_diff,avg_opp_loss_rate_at_venue_diff,avg_opp_loss_rate_at_venue_diff_diff,avg_opp_distance_km_change_diff,avg_opp_distance_km_change_diff_diff,avg_opp_avg_distance_km_change_diff,avg_opp_avg_distance_km_change_diff_diff,avg_opp_avg_elevation_meters_diff,avg_opp_avg_elevation_meters_diff_diff,avg_opp_elevation_meters_change_diff,avg_opp_elevation_meters_change_diff_diff,avg_opp_avg_elevation_meters_change_diff,avg_opp_avg_elevation_meters_change_diff_diff,avg_opp_avg_event_capacity_diff,avg_opp_avg_event_capacity_diff_diff,avg_opp_event_capacity_change_diff,avg_opp_event_capacity_change_diff_diff,avg_opp_avg_event_capacity_change_diff,avg_opp_avg_event_capacity_change_diff_diff,avg_opp_avg_event_attendance_diff,avg_opp_avg_event_attendance_diff_diff,avg_opp_event_attendance_change_diff,avg_opp_event_attendance_change_diff_diff,avg_opp_avg_event_attendance_change_diff,avg_opp_avg_event_attendance_change_diff_diff,avg_opp_avg_event_occupancy_pct_diff,avg_opp_avg_event_occupancy_pct_diff_diff,avg_opp_event_occupancy_pct_change_diff,avg_opp_event_occupancy_pct_change_diff_diff,avg_opp_avg_event_occupancy_pct_change_diff,avg_opp_avg_event_occupancy_pct_change_diff_diff
0,be38ed9ccfe2ee03,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,1,0.000251,0.000459,0.003968,0.006173,0.001980,0.004255,0.017857,0.022727,0.004762,0.010417,0.047619,0.032258,0.006061,0.012658,-23.666667,10,-0.013147,0.009568,-0.087911,-0.102858,-0.024253,...,8641.000000,-0.445987,0.000000,0.306207,-0.083333,0.166667,-0.083333,0.416667,-0.073333,0.438889,0.073333,-0.438889,1467.669156,-878.835479,215.908147,631.548674,-78.573333,59.241667,74.266667,39.433333,308.11666

In [21]:
# features["is_ppv"]
ppv_bout_ids = features.loc[features["is_ppv"] == 0, "id"].tolist()

temp = df.loc[(df["bout_id"].isin(ppv_bout_ids)) & (df["red_win"].notnull())].copy()
model_log_loss = log_loss(temp["red_win"], temp["Model"])
bovada_log_loss = log_loss(temp["red_win"], temp["Bovada"])
model_brier_score = brier_score_loss(temp["red_win"], temp["Model"])
bovada_brier_score = brier_score_loss(temp["red_win"], temp["Bovada"])

print(f"Model Log Loss (PPV): {model_log_loss:.6f}")
print(f"Bovada Log Loss (PPV): {bovada_log_loss:.6f}")
print(f"Delta: {model_log_loss - bovada_log_loss:.6f}")
print(f"Model Brier Score (PPV): {model_brier_score:.6f}")
print(f"Bovada Brier Score (PPV): {bovada_brier_score:.6f}")
print(f"Delta: {model_brier_score - bovada_brier_score:.6f}")

Model Log Loss (PPV): 0.614957
Bovada Log Loss (PPV): 0.614445
Delta: 0.000512
Model Brier Score (PPV): 0.213644
Bovada Brier Score (PPV): 0.213284
Delta: 0.000359


In [25]:
# features["is_female"]
female_bout_ids = features.loc[features["is_female"] == 1, "id"].tolist()

temp = df.loc[(df["bout_id"].isin(female_bout_ids)) & (df["red_win"].notnull())].copy()
model_log_loss = log_loss(temp["red_win"], temp["Model"])
bovada_log_loss = log_loss(temp["red_win"], temp["Bovada"])
model_brier_score = brier_score_loss(temp["red_win"], temp["Model"])
bovada_brier_score = brier_score_loss(temp["red_win"], temp["Bovada"])

print(f"Model Log Loss: {model_log_loss:.6f}")
print(f"Bovada Log Loss: {bovada_log_loss:.6f}")
print(f"Delta: {model_log_loss - bovada_log_loss:.6f}")
print(f"Model Brier Score: {model_brier_score:.6f}")
print(f"Bovada Brier Score: {bovada_brier_score:.6f}")
print(f"Delta: {model_brier_score - bovada_brier_score:.6f}")

Model Log Loss: 0.630458
Bovada Log Loss: 0.618340
Delta: 0.012118
Model Brier Score: 0.220585
Bovada Brier Score: 0.214636
Delta: 0.005949


In [26]:
import os
import sqlite3

db_path = os.path.join(os.path.dirname("__file__"), "..", "..", "data", "ufc.db")

In [45]:
query = """
SELECT id FROM ufcstats_bouts WHERE weight_class = 'Catch Weight'
"""

with sqlite3.connect(db_path) as conn:
    query_res = pd.read_sql_query(query, conn)
weight_class_bout_ids = query_res["id"].tolist()

temp = df.loc[(df["bout_id"].isin(weight_class_bout_ids)) & (df["red_win"].notnull())].copy()
model_log_loss = log_loss(temp["red_win"], temp["Model"])
bovada_log_loss = log_loss(temp["red_win"], temp["Bovada"])
model_brier_score = brier_score_loss(temp["red_win"], temp["Model"])
bovada_brier_score = brier_score_loss(temp["red_win"], temp["Bovada"])

print(f"Model Log Loss: {model_log_loss:.6f}")
print(f"Bovada Log Loss: {bovada_log_loss:.6f}")
print(f"Delta: {model_log_loss - bovada_log_loss:.6f}")
print(f"Model Brier Score: {model_brier_score:.6f}")
print(f"Bovada Brier Score: {bovada_brier_score:.6f}")
print(f"Delta: {model_brier_score - bovada_brier_score:.6f}")

Model Log Loss: 0.579667
Bovada Log Loss: 0.581412
Delta: -0.001745
Model Brier Score: 0.194415
Bovada Brier Score: 0.200654
Delta: -0.006239


In [57]:
query = """
WITH cte1 AS (
    SELECT fighter_id, t1.'order', bout_id FROM ufcstats_fighter_histories t1
    INNER JOIN bout_mapping t2 ON t1.bout_id = t2.ufcstats_id
),
cte2 AS (
    SELECT fighter_id, bout_id, ROW_NUMBER() OVER (PARTITION BY fighter_id ORDER BY t1.'order') AS ufc_order FROM cte1 t1
),
cte3 AS (
    SELECT
        id,
        t3.ufc_order AS red_ufc_order,
        t4.ufc_order AS blue_ufc_order
    FROM ufcstats_bouts t1
    INNER JOIN bout_mapping t2 ON t1.id = t2.ufcstats_id
    LEFT JOIN cte2 t3 ON t1.id = t3.bout_id AND t1.red_fighter_id = t3.fighter_id
    LEFT JOIN cte2 t4 ON t1.id = t4.bout_id AND t1.blue_fighter_id = t4.fighter_id
)
SELECT id FROM cte3 WHERE
    red_ufc_order > 5 AND blue_ufc_order > 5
"""

with sqlite3.connect(db_path) as conn:
    query_res = pd.read_sql_query(query, conn)
exp_level_bout_ids = query_res["id"].tolist()

temp = df.loc[(df["bout_id"].isin(exp_level_bout_ids)) & (df["red_win"].notnull())].copy()
model_log_loss = log_loss(temp["red_win"], temp["Model"])
bovada_log_loss = log_loss(temp["red_win"], temp["Bovada"])
model_brier_score = brier_score_loss(temp["red_win"], temp["Model"])
bovada_brier_score = brier_score_loss(temp["red_win"], temp["Bovada"])

print(f"Model Log Loss: {model_log_loss:.6f}")
print(f"Bovada Log Loss: {bovada_log_loss:.6f}")
print(f"Delta: {model_log_loss - bovada_log_loss:.6f}")
print(f"Model Brier Score: {model_brier_score:.6f}")
print(f"Bovada Brier Score: {bovada_brier_score:.6f}")
print(f"Delta: {model_brier_score - bovada_brier_score:.6f}")

Model Log Loss: 0.613856
Bovada Log Loss: 0.617388
Delta: -0.003532
Model Brier Score: 0.212762
Bovada Brier Score: 0.214226
Delta: -0.001464
